In [8]:
import sklearn
import pandas as pd
import numpy as np

from tensorflow import keras
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.regularizers import l2

In [9]:
COMMENT = 'comment_text'
LABELS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [10]:
base_path = 'jigsaw-toxic-comment-classification-challenge'
train = pd.read_csv(f'{base_path}/train.csv')
test = pd.read_csv(f'{base_path}/test.csv')
submission = pd.read_csv(f'{base_path}/sample_submission.csv')
test_labels = pd.read_csv(f'{base_path}/test_labels.csv')

# Problem understanding

# Dataset generation

# Metrics define

# Validation strategy

# Data processing (extract useful information)

In [11]:
MAX_TOKENS = 30000
MAX_SENTENSE_LEN = 100
EMBEDDING_SIZE = 300

In [12]:
x_train = train["comment_text"].values
y_train = train[LABELS].values
x_test = test["comment_text"].values

tokenizer = text.Tokenizer(num_words=MAX_TOKENS)
tokenizer.fit_on_texts(list(x_train) + list(x_test)) # is it a proper trick?

x_train, x_test = map(tokenizer.texts_to_sequences, [x_train, x_test])
x_train, x_test = map(lambda x: sequence.pad_sequences(x, maxlen=MAX_SENTENSE_LEN), [x_train, x_test])


# Data understanding & visualization

# De-noise (no drop data)

# Feature engineering

# Offline augmentation

# Standarization

# Scaling

# Normalization

# Feature selection

# Data selection

# Optimization

In [13]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, mask_zero=True, embeddings_regularizer=l2(1e-5))(model_input)
x = keras.layers.SpatialDropout1D(0.2)(x)
x = keras.layers.GRU(64, return_sequences=True, activation="relu", kernel_regularizer=l2(1e-5))(x)
x = keras.layers.SpatialDropout1D(0.2)(x)
x = keras.layers.GRU(32, return_sequences=False, activation="relu", kernel_regularizer=l2(1e-5))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

In [14]:
hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=5, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

Epoch 1/5


/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


593/593 [==============================] - 236s 398ms/step - loss: 0.1741 - acc: 0.6977 - val_loss: 0.0718 - val_acc: 0.9939
Epoch 2/5
593/593 [==============================] - 234s 395ms/step - loss: 0.0657 - acc: 0.9571 - val_loss: 0.0619 - val_acc: 0.9952
Epoch 3/5
593/593 [==============================] - 235s 396ms/step - loss: 0.0583 - acc: 0.9774 - val_loss: 0.0587 - val_acc: 0.9952
Epoch 4/5
593/593 [==============================] - 237s 399ms/step - loss: 0.0548 - acc: 0.9808 - val_loss: 0.0577 - val_acc: 0.9952
Epoch 5/5
593/593 [==============================] - 236s 399ms/step - loss: 0.0527 - acc: 0.9825 - val_loss: 0.0575 - val_acc: 0.9952


In [ ]:
y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('submission.csv', index=False)

# ~ 95.63%

# Parameter tuning

# Online augmentation

# Model selection / blending

# Post-processing

# Evaluation

# Reasoning

# Monitoring